In [1]:
import requests
from bs4 import BeautifulSoup

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [4]:
cookies = {
    '_ga': 'GA1.3.748161653.1700470356',
    '_gid': 'GA1.3.844182494.1700470356',
    '_ga_K3YJ93G42S': 'GS1.3.1700540517.2.0.1700540517.0.0.0',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7',
    'Cache-Control': 'no-cache',
    'Connection': 'keep-alive',
    # 'Cookie': '_ga=GA1.3.748161653.1700470356; _gid=GA1.3.844182494.1700470356; _ga_K3YJ93G42S=GS1.3.1700540517.2.0.1700540517.0.0.0',
    'Pragma': 'no-cache',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Google Chrome";v="119", "Chromium";v="119", "Not?A_Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

In [5]:
r = requests.get('https://www.dermatology.org.my/malaysia_journal.php', cookies = cookies, headers = headers)

In [9]:
soup = BeautifulSoup(r.content)
a = soup.find_all('a')
a = ['https://www.dermatology.org.my/' + a_.get('href') for a_ in a if a_.get('href').endswith('.pdf')]

In [11]:
urls_combined = sorted(list(set(a)))
len(urls_combined)

48

In [12]:
!mkdir dermatology.org.my

In [15]:
import os
from tqdm import tqdm

pdfs = urls_combined
for i in tqdm(range(len(pdfs))):
    filename = os.path.join('dermatology.org.my', f'{i}.pdf')
    if os.path.exists(filename):
        continue
        
    u = pdfs[i]
        
    r = requests.get(u)
    with open(filename, 'wb') as fopen:
        fopen.write(r.content)

100%|███████████████████████████████████████████| 48/48 [01:18<00:00,  1.63s/it]


In [16]:
from glob import glob
from tika import parser
import json

TIKA_HOST = 'http://localhost:9998'

In [17]:
files = glob('dermatology.org.my/*.pdf')

with open('dermatology.org.my.jsonl', 'w') as fopen:
    for f in tqdm(files):
        try:
            raw_xml = parser.from_file(f, TIKA_HOST, xmlContent=True)
            body = BeautifulSoup(raw_xml['content']).find('body')
            t = '\n'.join(body.findAll(text=True))
            fopen.write(f'{json.dumps(t)}\n')
        except:
            pass

100%|███████████████████████████████████████████| 48/48 [00:06<00:00,  7.74it/s]
